## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [274]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
import re
import collections

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns


Import functions that we'll need

In [141]:
# Cleaning Script
sw = stopwords.words("english")
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#", "'"}

def remove_stop(text, sw=sw) :
    # modify this function to remove stopwords
    return([ch for ch in text if ch not in sw])


def remove_punctuation(text, punct_set=tw_punct) : 
   text = [''.join(ch for ch in word if ch not in punct_set)
           for word in text]
   
   return(text)


def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    text = text.strip().split()
    return(text)



In [5]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

Note: A bit of reversal from last week's strategy. To deal with stopwords at the end of a sentence (e.g. He tossed it.), I removed punctuation except for apostrophes to save the contractions for stopword removal. This seemed to line up with the assertion a few chunks down. 

In [172]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions;
                            ''')

for row in query_results :

    # Run the text variable through cleaning steps
    text = row[0]
    text_low = text.lower()
    text_tok = tokenize(text_low)
    text_clean = remove_punctuation(text_tok)
    text_stop = remove_stop(text_clean)
    text_untok = " ".join(text_stop)
    
    # Store party as list
    party = row[1]
    
    
    # Create temp list
    temp_list = [text_untok, party]
    
    # Append to convention_data
    convention_data.append(temp_list)

Let's look at some random entries and see if they look right. 

In [173]:
random.choices(convention_data,k=10)

[['foreign prince', 'Republican'],
 ['reproductive justice', 'Democratic'],
 ['mission fight future equal ideals founders hopes children sacrifices veterans brave men women uniform families',
  'Democratic'],
 ['black americans standing native land probably represent oregon dual viruses covid19 racism laid bare equal healthcare access deaths communities color',
  'Democratic'],
 ['joe’s purpose always driven forward strength unstoppable faith unshakable it’s politicians political parties even it’s providence god faith us yes many classrooms quiet right playgrounds still listen closely hear sparks change air across country educators parents first responders americans walks life putting shoulders back fighting haven’t given need leadership worthy nation worthy honest leadership bring us back together recover pandemic prepare whatever else next dr',
  'Democratic'],
 ['he’ll love heart', 'Democratic'],
 ['rhode island ocean state restaurant fishing industry decimated pandemic lucky govern

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [190]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [191]:
def conv_features(text,fw = feature_words) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here

    # Args:
    text_tok = text.split()
    
    text_feat_words = [ch for ch in text_tok if ch in feature_words]


    
    
    ret_dict = dict.fromkeys(text_feat_words, True)
    
    return(ret_dict)

In [192]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [193]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [194]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [195]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [281]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

I found it very interesting that the most important features to the model were nearly all on the Republican side. Even going out to the 50 more important features, the only feature that favored Democrats was 'votes'. This is already a generic term that I'd expect both sides to use, so the model appears to be giving high weights to Republican features. This is likely the explanation to the low accuracy, as I think these words could be used in tweets by both sides. We'll further investigate these effects in the final section of the notebook.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [199]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [200]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

I'm going to go with the same puncation and stopword strategy. 

In [236]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results :

    # Run the text variable through cleaning steps
    text = row[2].decode('utf-8')
    text_low = text.lower()
    text_tok = tokenize(text_low)
    text_clean = remove_punctuation(text_tok)
    text_stop = remove_stop(text_clean)
    text_untok = " ".join(text_stop)

    # I'll also remove urls
    text_nourl = re.sub(r'http\S+', '', text_untok).strip()
    
    # Store party as list
    party = row[1]
    
    
    # Create temp list
    temp_list = [text_nourl, party]
    
    # Append to convention_data
    tweet_data.append(temp_list)

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [237]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [238]:

for tweet, party in tweet_data_sample :
    # Convert text to our feature word dictionary
    features = conv_features(text=tweet)

    # Run our classifier
    estimated_party = classifier.classify(featureset=features)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe #rallytogether
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater  #kag 🇺🇸
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr cavs tie series 22 i'm #allin216 repbarbaralee scared #roadtovictory
Actual party is Democra

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [242]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
  
    # Convert text to our feature word dictionary
    features = conv_features(text=tweet)

    # Run our classifier
    estimated_party = classifier.classify(featureset=features)

    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [248]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3694, 'Democratic': 678}),
             'Democratic': defaultdict(int,
                         {'Republican': 4727, 'Democratic': 903})})

In [250]:
# Actual Republicans
rep_correct = results.get('Republican').get('Republican')
rep_total = results.get('Republican').get('Democratic') + rep_correct
rep_acc = rep_correct/rep_total*100
print("The classifier predicted", round(rep_acc, 2), "percent of actual Republican tweets correctly")

# Actual Democrats
dem_correct = results.get('Democratic').get('Democratic')
dem_total = results.get('Democratic').get('Republican') + dem_correct
dem_acc = dem_correct/dem_total*100
print("The classifier predicted", round(dem_acc, 2), "percent of actual Democratic tweets correctly")

# Total
total_acc = (dem_correct + rep_correct)/num_to_score * 100
print("The classifier predicted", round(total_acc, 2), "percent of all the tweets correctly")

The classifier predicted 84.49 percent of actual Republican tweets correctly
The classifier predicted 16.04 percent of actual Democratic tweets correctly
The classifier predicted 45.97 percent of all the tweets correctly


Overfitting? We'll take a look at the accuracy on the convention train and test sets to see if we're overfitting.

In [256]:
print(round(nltk.classify.accuracy(classifier, train_set), 2))
print(nltk.classify.accuracy(classifier, test_set))

0.54
0.5


Are the samples unbalanced? 

In [280]:
parties = [x[1] for x in train_set]
collections.Counter(parties)

Counter({'Democratic': 1243, 'Republican': 798})

### Reflections

The results of both of the models paint a pretty clear picture. They were both pretty poor, with the convention data only having a 50% accuracy and the twitter data only ending up with a 46% accuracy. From the results dictionary, we saw that the model predicted 'Republican' for nearly all of the tweets, even for actual Democrats. So, at first I investigated if the model was potentially overfit by comparing the train and test accuracy of our convention data (since we used it to train the data). Our test accuracy was only 50%, not much better than the training accuracy. I then investigated if maybe there was a sample imbalance of our convention data, thinking maybe the classifier was optimizing accuracy and our training set was made of nearly all Republican speeches. However, our training data actually had more Democrat texts. Finally, I thought there were plenty of features in our feature words to capture everything. 

So, my suggestion to improve the classifier would turn to a different model beside Naive Bayes. Likely a non-linear model to help us capture a little bit more of the variance. 